In [17]:
import os
import subprocess as sp
import yaml
import shutil
from tqdm import tqdm

# Convert notebooks to html

In [2]:
os.chdir('/Users/ginoprasad/ginoprasad.github.io')

In [3]:
metadata_path = 'metadata.yaml'
with open(metadata_path) as infile:
    metadata = yaml.safe_load(infile)

In [4]:
for project_notebook_path in metadata['Projects'][:]:
    if not os.path.exists(project_notebook_path):
        print(f"REMOVING {project_notebook_path}")
        metadata['Projects'].remove(project_notebook_path)
        with open(metadata_path, 'w') as outfile:
            yaml.dump(metadata, outfile, default_flow_style=False)

In [5]:
temp_path = f'{os.getcwd()}/projects/temp.html'

In [6]:
max_base_filename_length = 50

In [7]:
temp_path

'/Users/ginoprasad/ginoprasad.github.io/projects/temp.html'

In [8]:
project_notebook_path

'/Users/ginoprasad/ai_stuff/edge_detector.ipynb'

In [23]:
project_names, project_paths = [], []
for project_notebook_path in tqdm(metadata['Projects']):

    print(project_notebook_path)
    title_line = sp.run(f"head -n 20 {project_notebook_path}", shell=True, capture_output=True).stdout.decode().split('\n')
    print(title_line)
    title_line = ipynb_yaml["cells"][0]["source"][0]
    project_name = title_line.strip().lstrip('# ')
    
    project_base_path = os.path.basename(project_notebook_path)[:-len('.ipynb')]
    while len(project_base_path) > max_base_filename_length:
        project_base_path = ' '.join(project_base_path.split(' ')[:-1])
    if not project_base_path:
        print(f"\n\n\n\n\tWarning: Project '{project_name}' Name exceeds recommended length\n\n\n\n")
        project_base_path = project_name
    project_path = f'{os.getcwd()}/projects/{project_base_path}.html'
    
    if os.path.getmtime(project_path) > os.path.getmtime(project_notebook_path):
        continue
    
    print(project_base_path)
    print(f"Converting {project_notebook_path}")
    sp.run(f"jupyter nbconvert --to html '{project_notebook_path}' --output '{temp_path}'", shell=True)
    print(f'Project Name: {project_name}')

    with open(temp_path) as infile:
        lines = infile.readlines()


    title = ' '.join(map(lambda x: x[0].upper() + x[1:] if x else x, project_base_path.split('_')))
    lines[5] = lines[5][:len('<title>')] + title + lines[5][lines[5].index('</title>'):]
 
    with open(temp_path, 'w') as outfile:
        lines.insert(5, '<link rel="icon" href="../docs/assets/logo.png"><iframe src="../header.html" style="height: 12rem; width: 100%" frameborder="0" scrolling="no"></iframe>\n')
        outfile.write(''.join(lines))
    
    assert project_path not in project_paths
    os.rename(temp_path, project_path)
    
    project_names.append(project_name)
    project_paths.append(project_path)
    print('\n')

 35%|█████████▋                  | 8/23 [00:00<00:00, 38.18it/s]

/Users/ginoprasad/ai_stuff/earth_movers_distance.ipynb
['{', ' "cells": [', '  {', '   "cell_type": "markdown",', '   "id": "9f16d75b",', '   "metadata": {},', '   "source": [', '    "# Earth Mover\'s Distance for NLP using Network Simplex\\n",', '    "<br>\\n",', '    "\\n",', '    "> #### Gino Prasad\\n",', '    "> #### 07/24/23\\n"', '   ]', '  },', '  {', '   "cell_type": "markdown",', '   "id": "3433b04d",', '   "metadata": {},', '   "source": [', '    "# Preview"', '']
/Users/ginoprasad/ai_stuff/chess_bot/mcst_alphazero.ipynb
['{', ' "cells": [', '  {', '   "cell_type": "markdown",', '   "id": "40e810ea",', '   "metadata": {},', '   "source": [', '    "# Deep Learning with Chess Monte Carlo Search Tree"', '   ]', '  },', '  {', '   "cell_type": "markdown",', '   "id": "999233ef",', '   "metadata": {},', '   "source": [', '    "## Defining Graph"', '   ]', '  },', '  {', '   "cell_type": "code",', '']
/Users/ginoprasad/ai_stuff/Translation/transformer_seq_to_seq.ipynb
['{', ' "cel

 70%|██████████████████▊        | 16/23 [00:00<00:00, 38.90it/s]

['{', ' "cells": [', '  {', '   "cell_type": "markdown",', '   "id": "81635586",', '   "metadata": {},', '   "source": [', '    "# Inner Product in Hilbert Space\\n",', '    "<br>\\n",', '    "\\n",', '    "> #### Gino Prasad\\n",', '    "> #### 05/08/23\\n"', '   ]', '  },', '  {', '   "cell_type": "markdown",', '   "id": "57efea84",', '   "metadata": {},', '   "source": [', '    "https://www.youtube.com/watch?v=3N2vN76E-QA&ab_channel=QuantumSense"', '']
/Users/ginoprasad/ai_stuff/autotune_phase_vocoder/Autotune_MUS_15.ipynb
['{', ' "cells": [', '  {', '   "cell_type": "markdown",', '   "id": "1865fd35",', '   "metadata": {},', '   "source": [', '    "# MUS 15 Project 1: Creating Autotune From Scratch Using Phase Vocoders"', '   ]', '  },', '  {', '   "cell_type": "markdown",', '   "id": "3c492546",', '   "metadata": {},', '   "source": [', '    "Gino Prasad, Saba Heydari Seradj, Ashish Dalvi"', '   ]', '  },', '  {', '   "cell_type": "markdown",', '']
/Users/ginoprasad/ai_stuff/autot

100%|███████████████████████████| 23/23 [00:00<00:00, 38.82it/s]

['']
/Users/ginoprasad/ai_stuff/text_classifier/Handwriten Text Parser.ipynb
['']
/Users/ginoprasad/ai_stuff/Gaussian Blur Test.ipynb
['']
/Users/ginoprasad/ai_stuff/audio_transcriber/play_note.ipynb
['{', ' "cells": [', '  {', '   "cell_type": "markdown",', '   "id": "37bfc694",', '   "metadata": {},', '   "source": [', '    "# Pitch Reconstruction with Audio Processing"', '   ]', '  },', '  {', '   "cell_type": "code",', '   "execution_count": 2,', '   "id": "559c13b5",', '   "metadata": {},', '   "outputs": [],', '   "source": [', '    "from IPython.display import Audio"', '   ]', '  },', '']
/Users/ginoprasad/Wordle/wordle_bot.ipynb
['{', ' "cells": [', '  {', '   "cell_type": "markdown",', '   "id": "79ad2763",', '   "metadata": {},', '   "source": [', '    "# Wordle Bot\\n",', '    "<br>\\n",', '    "\\n",', '    "> #### Gino Prasad\\n",', '    "> #### 07/05/22\\n"', '   ]', '  },', '  {', '   "cell_type": "markdown",', '   "id": "39844eac",', '   "metadata": {},', '   "source": 

In [10]:
title_line

"# Earth Mover's Distance for NLP using Network Simplex\n"

In [10]:
index_html_path = 'index.html'
index_html_lines = open(index_html_path).readlines()

In [11]:
publications_list_index_start = ["Publications" in x for x in index_html_lines].index(True) + 2
publications_list_index_end = index_html_lines[publications_list_index_start:].index('\t\t</ul>\n') + publications_list_index_start

publications_list = []
for publication in metadata['Publications']:
    name = publication['name']
    publications_list.append(f'\t\t\t<li>\n\t\t\t\t<p>{name}<p>\n\t\t\t\t<h3>&emsp;&emsp;{publication["journal"]}</h3>\n\t\t\t\t&emsp;&emsp;&emsp;&emsp;<a href="{publication["doi"]}">{publication["doi"]}</a>\n\t\t\t</li>\n')
index_html_lines = index_html_lines[:publications_list_index_start] + publications_list + index_html_lines[publications_list_index_end:]

In [12]:
project_list_index_start = ["Cool Projects" in x for x in index_html_lines].index(True) + 2
project_list_index_end = index_html_lines[project_list_index_start:].index('\t\t</ul>\n') + project_list_index_start

new_project_list =  [f'\t\t\t<li><a href="projects/{os.path.basename(html_path)}">{name}</a></li>\n' for name, html_path in zip(project_names, project_paths)]
index_html_lines = index_html_lines[:project_list_index_start] + new_project_list + index_html_lines[project_list_index_end:]
index_html_lines[project_list_index_start-2] = f"\t\t<h2> Cool Projects ({len(metadata['Projects'])}) </h2>\n"

# Copying CV and Updating Links

In [13]:
assert shutil.copy(metadata['CV'], f"projects/{os.path.basename(metadata['CV'])}")

In [14]:
tag_dict = {tag: metadata[tag] for tag in ['CV', 'LinkedIn', 'GitHub']}
tag_dict['CV'] = f"projects/{os.path.basename(tag_dict['CV'])}"

In [15]:
for i, line in enumerate(index_html_lines):
    for tag in tag_dict:
        prefix = f"<a id='{tag}' href='"
        if line.startswith(prefix):
            print(line.strip())
            new_line = prefix + tag_dict[tag] + line[len(prefix) + line[len(prefix):].index("'"):]
            print(new_line)
            index_html_lines[i] = new_line
    
    if line.startswith(prefix):
        del tag_dict[tag]

# Writing Updated Index File

In [16]:
with open(index_html_path, 'w') as outfile:
    outfile.write(''.join(index_html_lines))

In [17]:
sp.run(f"cd '{os.getcwd()}'; git add .; git commit -m 'Automated Website Update'; git push origin main", shell=True)

[main ddf8b06] Automated Website Update
 2 files changed, 28 insertions(+), 429 deletions(-)


remote: This repository moved. Please use the new location:        
remote:   https://github.com/GinoP123/ginoprasad.github.io.git        
To https://github.com/ginoprasad/ginoprasad.github.io
   cb205d6..ddf8b06  main -> main


CompletedProcess(args="cd '/Users/ginoprasad/ginoprasad.github.io'; git add .; git commit -m 'Automated Website Update'; git push origin main", returncode=0)

# Updating Python Script

In [18]:
if hasattr(__builtins__,'__IPYTHON__'):
    sp.run(f"jupyter nbconvert --to script 'update_website.ipynb' --output 'update_website'", shell=True)

[NbConvertApp] Converting notebook update_website.ipynb to script
[NbConvertApp] Writing 5177 bytes to update_website.py
